In [26]:
import tensorflow as tf
import pandas as pd
import nltk


import csv

In [50]:

#Organizando a base de dados em um arquivo csv

df_txt = pd.read_csv('base.txt', delimiter='\t', header=None)

nome_arquivo_csv = "base.csv"

with open(nome_arquivo_csv, mode='w', newline='',encoding='utf-8') as arquivo_csv:
    escritor_csv = csv.writer(arquivo_csv, delimiter=',', quotechar='"',
                              quoting=csv.QUOTE_MINIMAL)
    # Escrever as linhas no arquivo CSV
    for index, row in df_txt.iterrows():
        texto = row[0].replace("(", "").replace(")", "").replace("'", "")
        #print(texto)
        posicao_ultima_virgula = texto.rfind(',')
        if posicao_ultima_virgula != -1:
            dados = texto[:posicao_ultima_virgula] + texto[posicao_ultima_virgula+1:]
            #print(dados)
            dados = dados.strip()
        escritor_csv.writerow(dados.split(','))
print(f"Arquivo CSV '{nome_arquivo_csv}' criado com sucesso!")


Arquivo CSV 'base.csv' criado com sucesso!
